### 1.创新性的案例
#### 完成课程实践的基础上，重新自己起一个Notebook，有创新点
#### 基于LangChain+ChatGLM3的本地知识库问答



##### 实现了基于LangChain+ChatGLM3的本地知识库问答，本地知识库支持txt、md、pdf、jpg文件。

## 2. 案例体验
🔹 本案例需使用 P100 及以上规格运行，请确保运行规格一致，可按照下图切换规格。

![](https://modelarts-labs-bj4-v2.obs.cn-north-4.myhuaweicloud.com/case_zoo/chatglm3/image/1.png)

🔹 点击Run in ModelArts，将会进入到ModelArts CodeLab中，这时需要你登录华为云账号，如果没有账号，则需要注册一个，且要进行实名认证，参考[《ModelArts准备工作_简易版》](https://developer.huaweicloud.com/develop/aigallery/article/detail?id=4ce709d6-eb25-4fa4-b214-e2e5d6b7919c) 即可完成账号注册和实名认证。 登录之后，等待片刻，即可进入到CodeLab的运行环境

🔹 出现 Out Of Memory ，请检查是否为您的参数配置过高导致，修改参数配置，重启kernel或更换更高规格资源进行规避❗❗❗

### 2.1 下载模型和数据

下载nltk_data数据

In [1]:
import os
import moxing as mox

work_dir = '/home/ma-user/work'
obs_path = 'obs://dtse-models/tar-models/nltk_data.tar'
ma_path = os.path.join(work_dir, 'nltk_data.tar')
mox.file.copy(obs_path, ma_path)

mox.file.copy_parallel('obs://modelarts-labs-bj4-v2/case_zoo/langchain-ChatGLM/file/docs','/home/ma-user/work/docs')

INFO:root:Using MoXing-v2.1.0.5d9c87c8-5d9c87c8
INFO:root:Using OBS-Python-SDK-3.20.9.1


进入nltk_data目录，解压数据压缩包

In [2]:
os.chdir(work_dir)
!pwd
!tar -xvf nltk_data.tar

/home/ma-user/work
nltk_data/
nltk_data/misc/
nltk_data/misc/mwa_ppdb.zip
nltk_data/misc/perluniprops.xml
nltk_data/misc/mwa_ppdb.xml
nltk_data/misc/perluniprops.zip
nltk_data/tokenizers/
nltk_data/tokenizers/punkt/
nltk_data/tokenizers/punkt/french.pickle
nltk_data/tokenizers/punkt/polish.pickle
nltk_data/tokenizers/punkt/.DS_Store
nltk_data/tokenizers/punkt/portuguese.pickle
nltk_data/tokenizers/punkt/german.pickle
nltk_data/tokenizers/punkt/swedish.pickle
nltk_data/tokenizers/punkt/malayalam.pickle
nltk_data/tokenizers/punkt/estonian.pickle
nltk_data/tokenizers/punkt/finnish.pickle
nltk_data/tokenizers/punkt/spanish.pickle
nltk_data/tokenizers/punkt/PY3/
nltk_data/tokenizers/punkt/PY3/french.pickle
nltk_data/tokenizers/punkt/PY3/polish.pickle
nltk_data/tokenizers/punkt/PY3/portuguese.pickle
nltk_data/tokenizers/punkt/PY3/german.pickle
nltk_data/tokenizers/punkt/PY3/swedish.pickle
nltk_data/tokenizers/punkt/PY3/malayalam.pickle
nltk_data/tokenizers/punkt/PY3/estonian.pickle
nltk_data

下载text2vec-large-chinese模型，用于中文通用语义匹配

In [3]:
import os
import moxing as mox

obs_path = 'obs://dtse-models/tar-models/text2vec-large-chinese.tar'
ma_path = os.path.join(work_dir, 'text2vec-large-chinese.tar')
mox.file.copy(obs_path, ma_path)

进入text2vec-large-chinese目录，解压模型压缩包

In [4]:
os.chdir(work_dir)
!pwd
!tar -xvf text2vec-large-chinese.tar

/home/ma-user/work
text2vec-large-chinese/
text2vec-large-chinese/.gitattributes
text2vec-large-chinese/README.md
text2vec-large-chinese/config.json
text2vec-large-chinese/eval_results.txt
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/blobs/
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/blobs/eaf5cb71c0eeab7db3c5171da504e5867b3f67a78e07bdba9b52d334ae35adb3.lock
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/refs/
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/refs/main
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/snapshots/
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/snapshots/064717f2acfd7253bea91079d59b82e50b58c886/
text2vec-large-chinese/pytorch_model.bin
text2vec-large-chinese/special_tokens_map.json
text2vec-large-chinese/tmpqlu9nxcm
text2vec-large-chinese/tokenizer.json
text2vec-large-chi

### 2.2 配置环境

本案例依赖Python3.10.10及以上环境，因此我们首先创建虚拟环境：

In [1]:
!/home/ma-user/anaconda3/bin/conda create -n python-3.10.10 python=3.10.10 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.10.10/bin/pip install ipykernel

/home/ma-user/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /home/ma-user/anaconda3/envs/python-3.10.10

  added / updated specs:
    - python=3.10.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           3 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    _openmp_mutex-5.1          |            1_gnu          21 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    bzip2-1.0.8                |       h7b6447c_0          78 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    ca-certif

In [2]:
import json
import os

data = {
   "display_name": "python-3.10.10",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.10.10/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/anaconda3/envs/PyTorch-1.8/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.10.10/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

创建完成后，稍等片刻，或刷新页面，点击右上角kernel选择python-3.10.10

![](https://modelarts-labs-bj4-v2.obs.cn-north-4.myhuaweicloud.com/case_zoo/chatglm3/image/2.png)

### 2.3 安装依赖库

In [1]:
!pip install transformers==4.30.2
!pip install sentencepiece==0.1.99
!pip install torch==2.0.1
!pip install markdown==3.4.3
!pip install faiss-gpu==1.7.2
!pip install langchain==0.0.329 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install nltk==3.8.1 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install unstructured==0.10.24 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install sentence-transformers==2.2.2

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 84.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 19.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 18.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 10.0 MB/s eta 0:00:

## 3. 中文文字匹配

langchain支持很多中embedding模型，例如[text2vec-large-chinese](https://github.com/shibing624/text2vec)、[m3e-large](https://github.com/wangyingdong/m3e-base)、[bge-large-zh](https://github.com/jsonzhuwei/bge-large-zh)等，本文中使用text2vec-large-chinese模型实现。

我们将待匹配的文字通过text2vec-large-chinese模型，转成嵌入向量，然后计算两个向量直接的相似性来进行匹配。

In [2]:
import os
import nltk
work_dir = '/home/ma-user/work'
docs_path = os.path.join(work_dir, 'docs')
nltk.data.path.append(os.path.join(work_dir, 'nltk_data'))

import numpy as np
from nltk import data
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter,MarkdownTextSplitter
from langchain.document_loaders import UnstructuredFileLoader,UnstructuredMarkdownLoader

In [3]:
embedding_model = 'text2vec-large-chinese'

#基于余弦相似性公式计算两个向量之间的相似度
def get_cos_similar(v1: list, v2: list):
    num = float(np.dot(v1, v2))  # 向量点乘
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)  # 求模长的乘积
    return 0.5 + 0.5 * (num / denom) if denom != 0 else 0

#加载text2vec-large-chinese模型
def load_embeddings():
    embedding_model_path = os.path.join(work_dir, embedding_model)
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_path)
    return embeddings

#计算两段文字的相似度
def get_embedding_sim(s1, s2, embeddings):
    embedding1 = embeddings.embed_query(s1)#将文本转为向量
    print('embedding1: ', len(embedding1))
    embedding2 = embeddings.embed_query(s2)
    sim = get_cos_similar(embedding1, embedding2)
    print('sim of \'{0}\' and \'{1}\' is : {2}'.format(s1, s2, sim))
    return sim

In [4]:
sentence1 = "我今天心情很差"
sentence2 = "我今天很不开心"
sentence3 = "what are you弄啥嘞"
embeddings = load_embeddings()
get_embedding_sim(sentence1, sentence2, embeddings)
get_embedding_sim(sentence1, sentence3, embeddings)

/home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/ma-user/work/text2vec-large-chinese. Creating a new one with MEAN pooling.


embedding1:  1024
sim of '我今天心情很差' and '我今天很不开心' is : 0.8813112714604454
embedding1:  1024
sim of '我今天心情很差' and 'what are you弄啥嘞' is : 0.5937208396102376


0.5937208396102376

## 4. 文档查询
对于文档查询，我们首先也是将分割后的文档转成嵌入向量，然后存储到向量数据库，再根据查询条件，从向量数据库进行搜索。

langchain支持的向量数据库有很多种，例如：[FAISS](https://github.com/facebookresearch/faiss)、[Milvus](https://github.com/milvus-io/milvus)、[PGVector](https://github.com/pgvector/pgvector)等，本文使用的是FAISS。

In [5]:
#加载txt文件
def load_txt_file(txt_file):    
    loader = UnstructuredFileLoader(os.path.join(work_dir, txt_file))
    docs = loader.load()
    return docs

#加载md文件
def load_md_file(md_file):    
    loader = UnstructuredMarkdownLoader(os.path.join(work_dir, md_file))
    docs = loader.load()
    return docs

#分割txt文件
def load_txt_splitter(txt_file, chunk_size=100, chunk_overlap=20):
    docs = load_txt_file(txt_file)
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = text_splitter.split_documents(docs)
    return split_docs

#分割md文件
def load_md_splitter(md_file, chunk_size=100, chunk_overlap=20):
    docs = load_md_file(md_file)
    text_splitter = MarkdownTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = text_splitter.split_documents(docs)
    return split_docs

#分割docs_path目录下的文件，并将其转为向量，放到FAISS向量数据库中
def load_vector_store(docs_path):
    split_docs = []
    for doc in os.listdir(docs_path):
        doc_path = f'{docs_path}/{doc}'
        if doc_path.endswith('.txt'):
            docs = load_txt_splitter(doc_path)
            split_docs.extend(docs)
        elif doc_path.endswith('.md'):
            docs = load_md_splitter(doc_path)
            split_docs.extend(docs)
        else:
            print('不支持的文件类型:', doc_path)
            continue
    embeddings = load_embeddings()
    vector_store = FAISS.from_documents(split_docs, embeddings)
    return vector_store

#从向量数据集进行内容查询
def sim_search(query, vector_store):
    #similarity_search_with_score返回相似的文档内容和查询与文档的距离分数
    #返回的距离分数是L2距离。因此，得分越低越好。
    re = vector_store.similarity_search_with_score(query)
    print('query result: ', re)

In [6]:
query = "ModelBox支持哪两种方式"
vector_store = load_vector_store(docs_path)
sim_search(query, vector_store)

不支持的文件类型: /home/ma-user/work/docs/.ipynb_checkpoints


Created a chunk of size 146, which is longer than the specified 100
No sentence-transformers model found with name /home/ma-user/work/text2vec-large-chinese. Creating a new one with MEAN pooling.


不支持的文件类型: /home/ma-user/work/docs/modelbox.jpg
不支持的文件类型: /home/ma-user/work/docs/modelbox.pdf
query result:  [(Document(page_content='ModelBox支持两种方式运行，一种是服务化，一种是SDK，开发者可以按照下表选择相关的开发模式。', metadata={'source': '/home/ma-user/work/docs/modelbox.txt'}), 420.72437), (Document(page_content='2. SDK：ModelBox提供了ModelBox开发库，使用于扩展现有应用支持高性能AI推理，专注AI推理业务，支持c++，Python集成', metadata={'source': '/home/ma-user/work/docs/modelbox.txt'}), 587.23193), (Document(page_content='如果是第一次创建工程，在ModelBox', metadata={'source': '/home/ma-user/work/docs/第一个应用.md'}), 598.4208), (Document(page_content='也就是说，ModelBox的Pipeline模式，首先需要将应用的流程图构建出来，再分别实现图中的每个模块（ModelBox中称为“功能单元”），对于上面的视频应用：读取摄像头并输出原始画面，对应的', metadata={'source': '/home/ma-user/work/docs/第一个应用.md'}), 646.28265)]


### 创新性的案例
#### 完成课程实践的基础上，重新自己起一个Notebook，有创新点
#### 基于LangChain+ChatGLM3的本地知识库问答

#### 2.1 下载ChatGLM3模型

In [1]:
import os
import moxing as mox

#拷贝chatglm3模型到chatglm3-6b目录下
work_dir = '/home/ma-user/work'
obs_path = 'obs://dtse-models/tar-models/chatglm3-6b.tar'
ma_path = os.path.join(work_dir, 'chatglm3-6b.tar')
mox.file.copy(obs_path, ma_path)

mox.file.copy_parallel('obs://modelarts-labs-bj4-v2/case_zoo/langchain-ChatGLM/file/service','service' )
mox.file.copy_parallel('obs://modelarts-labs-bj4-v2/course/ModelBox/frpc_linux_amd64', '/home/ma-user/work/frpc_linux_amd64')
# mox.file.copy_parallel('obs://modelarts-labs-bj4-v2/course/ModelBox/frpc_linux_amd64', '/home/ma-user/work/frpc_linux_amd64')

INFO:root:Using MoXing-v2.1.0.5d9c87c8-5d9c87c8
INFO:root:Using OBS-Python-SDK-3.20.9.1


进入chatglm3目录，解压模型压缩包

In [2]:
os.chdir(work_dir)
!pwd
!tar -xvf chatglm3-6b.tar

/home/ma-user/work
chatglm3-6b/
chatglm3-6b/.gitattributes
chatglm3-6b/MODEL_LICENSE
chatglm3-6b/README.md
chatglm3-6b/config.json
chatglm3-6b/configuration_chatglm.py
chatglm3-6b/modeling_chatglm.py
chatglm3-6b/pytorch_model-00001-of-00007.bin
chatglm3-6b/pytorch_model-00002-of-00007.bin
chatglm3-6b/pytorch_model-00003-of-00007.bin
chatglm3-6b/pytorch_model-00004-of-00007.bin
chatglm3-6b/pytorch_model-00005-of-00007.bin
chatglm3-6b/pytorch_model-00006-of-00007.bin
chatglm3-6b/pytorch_model-00007-of-00007.bin
chatglm3-6b/pytorch_model.bin.index.json
chatglm3-6b/quantization.py
chatglm3-6b/tokenization_chatglm.py
chatglm3-6b/tokenizer.model
chatglm3-6b/tokenizer_config.json


下载nltk_data数据

In [3]:
import os
import moxing as mox

work_dir = '/home/ma-user/work'
obs_path = 'obs://dtse-models/tar-models/nltk_data.tar'
ma_path = os.path.join(work_dir, 'nltk_data.tar')
mox.file.copy(obs_path, ma_path)

mox.file.copy_parallel('obs://modelarts-labs-bj4-v2/case_zoo/langchain-ChatGLM/file/docs','/home/ma-user/work/docs')

进入nltk_data目录，解压数据压缩包

In [4]:
os.chdir(work_dir)
!pwd
!tar -xvf nltk_data.tar

/home/ma-user/work
nltk_data/
nltk_data/misc/
nltk_data/misc/mwa_ppdb.zip
nltk_data/misc/perluniprops.xml
nltk_data/misc/mwa_ppdb.xml
nltk_data/misc/perluniprops.zip
nltk_data/tokenizers/
nltk_data/tokenizers/punkt/
nltk_data/tokenizers/punkt/french.pickle
nltk_data/tokenizers/punkt/polish.pickle
nltk_data/tokenizers/punkt/.DS_Store
nltk_data/tokenizers/punkt/portuguese.pickle
nltk_data/tokenizers/punkt/german.pickle
nltk_data/tokenizers/punkt/swedish.pickle
nltk_data/tokenizers/punkt/malayalam.pickle
nltk_data/tokenizers/punkt/estonian.pickle
nltk_data/tokenizers/punkt/finnish.pickle
nltk_data/tokenizers/punkt/spanish.pickle
nltk_data/tokenizers/punkt/PY3/
nltk_data/tokenizers/punkt/PY3/french.pickle
nltk_data/tokenizers/punkt/PY3/polish.pickle
nltk_data/tokenizers/punkt/PY3/portuguese.pickle
nltk_data/tokenizers/punkt/PY3/german.pickle
nltk_data/tokenizers/punkt/PY3/swedish.pickle
nltk_data/tokenizers/punkt/PY3/malayalam.pickle
nltk_data/tokenizers/punkt/PY3/estonian.pickle
nltk_data

下载text2vec-large-chinese模型，用于中文通用语义匹配

In [2]:
import os
import moxing as mox

work_dir = '/home/ma-user/work'

obs_path = 'obs://dtse-models/tar-models/text2vec-large-chinese.tar'
ma_path = os.path.join(work_dir, 'text2vec-large-chinese.tar')
mox.file.copy(obs_path, ma_path)

进入text2vec-large-chinese目录，解压模型压缩包

In [3]:
os.chdir(work_dir)
!pwd
!tar -xvf text2vec-large-chinese.tar

/home/ma-user/work
text2vec-large-chinese/
text2vec-large-chinese/.gitattributes
text2vec-large-chinese/README.md
text2vec-large-chinese/config.json
text2vec-large-chinese/eval_results.txt
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/blobs/
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/blobs/eaf5cb71c0eeab7db3c5171da504e5867b3f67a78e07bdba9b52d334ae35adb3.lock
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/refs/
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/refs/main
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/snapshots/
text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese/snapshots/064717f2acfd7253bea91079d59b82e50b58c886/
text2vec-large-chinese/pytorch_model.bin
text2vec-large-chinese/special_tokens_map.json
text2vec-large-chinese/tmpqlu9nxcm
text2vec-large-chinese/tokenizer.json
text2vec-large-chi

### 2.2 配置环境

本案例依赖Python3.10.10及以上环境，因此我们首先创建虚拟环境：

In [1]:
!/home/ma-user/anaconda3/bin/conda create -n python-3.10.10 python=3.10.10 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.10.10/bin/pip install ipykernel

/home/ma-user/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /home/ma-user/anaconda3/envs/python-3.10.10

  added / updated specs:
    - python=3.10.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           3 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    _openmp_mutex-5.1          |            1_gnu          21 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    bzip2-1.0.8                |       h7b6447c_0          78 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    ca-certif

In [3]:
import json
import os

data = {
   "display_name": "python-3.10.10",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.10.10/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/anaconda3/envs/PyTorch-1.8/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.10.10/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

### 2.3 安装依赖库

In [1]:
!pip install transformers==4.30.2
!pip install sentencepiece==0.1.99
!pip install torch==2.0.1
!pip install markdown==3.4.3
!pip install faiss-gpu==1.7.2
!pip install langchain==0.0.329 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install nltk==3.8.1 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install unstructured==0.10.24 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install sentence-transformers==2.2.2
!cp /home/ma-user/work/frpc_linux_amd64 /home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2
!chmod +x /home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 98.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 63.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 39.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 15.0 MB/s eta 0:00:00

In [2]:
%pip install rapidocr_onnxruntime pdf2image pdfminer.six mdtex2html gradio==3.39 -i https://pypi.tuna.tsinghua.edu.cn/simple --trusted-host pypi.tuna.tsinghua.edu.cn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 83.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.7 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 83.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 99.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/13

## 3. 代码开发

In [1]:
from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate

from service.config import LangChainCFG
from service.chatglm_service import ChatGLMService
# from service.chatglm_service import ChatGLMService,ChatGLMForConditionalGeneration
from service.knowledge_service import KnowledgeService


class LangChainApplication(object):
    #初始化，加载ChatGLM3模型和本地知识库
    def __init__(self,config):
        self.config = config
        self.llm_service = ChatGLMService()
        self.llm_service.load_model(model_name_or_path=self.config.llm_model_path)
        self.knowledge_service = KnowledgeService(config)
    #获取大语言模型返回的答案（基于本地知识库查询）
    def get_knowledeg_based_answer(self,query,
                                  history_len=5,
                                  temperature=0.1,
                                  top_p=0.9,
                                  top_k=4,
                                  chat_history=[]):
        #定义查询的提示模板格式：
        prompt_template = """
基于以下已知信息，简洁和专业的来回答用户的问题。
如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
已知内容:
{context}
问题:
{question}
    """
        prompt = PromptTemplate(template=prompt_template,
                            input_variables=["context", "question"])
        self.llm_service.history = chat_history[-history_len:] if history_len>0 else []
        self.llm_service.temperature = temperature
        self.llm_service.top_p = top_p
        
        #利用预先存在的语言模型、检索器来创建并初始化BaseRetrievalQA类的实例
        knowledge_chain = RetrievalQA.from_llm(
        llm = self.llm_service,
        #基于本地知识库构建一个检索器，并仅返回top_k的结果
        retriever = self.knowledge_service.knowledge_base.as_retriever(
        search_kwargs={"k":top_k}),
        prompt = prompt)
        #combine_documents_chain的作用是将查询返回的文档内容（page_content）合并到一起作为prompt中context的值
        #将combine_documents_chain的合并文档内容改为{page_content}
        
        knowledge_chain.combine_documents_chain.document_prompt = PromptTemplate(
        input_variables=["page_content"],template="{page_content}")
        
        #返回结果中是否包含源文档
        knowledge_chain.return_source_documents = True
        
        #传入问题内容进行查询
        result = knowledge_chain({"query":query})
        return result
    
    #获取大语言模型返回的答案（未基于本地知识库查询）
    def get_llm_answer(self,query):
        result = self.llm_service._call(query)
        return result
    
    
    

/home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from service.config import LangChainCFG

config = LangChainCFG()
application = LangChainApplication(config)

result = application.get_llm_answer('ModelBox是什么')
print('\nresult of llm:\n')
print(result)
print('\n#############################################\n')

application.knowledge_service.init_knowledge_base()
result2 = application.get_knowledeg_based_answer('ModelBox是什么')
print('\n############################################\n')
print('\nresult of knowledge base:\n')
print(result2)

Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.26s/it]
No sentence-transformers model found with name /home/ma-user/work/text2vec-large-chinese. Creating a new one with MEAN pooling.



result of llm:

ModelBox是一个基于Web的建模平台，它允许用户创建、编辑和分享各种类型的模型，包括数据模型、机器学习模型、业务流程模型等。ModelBox旨在为用户提供一个易于使用的界面，以便他们可以快速地创建和管理模型，而无需深入了解代码或复杂的建模技术。通过ModelBox，用户可以轻松地将模型部署到生产环境中，并与其他人共享和协作。ModelBox的主要特点是易于使用、功能丰富、灵活性和可扩展性。

#############################################


#####init_knowledge_base#####

modelbox.jpg
modelbox.pdf
modelbox.txt
第一个应用.md

############################################


result of knowledge base:

{'query': 'ModelBox是什么', 'result': 'ModelBox是一个AI应用开发平台，提供了一种快速、高效、灵活的方式，让开发者能够构建、部署和管理AI应用。ModelBox具有以下特点：\n\n1. 基于云的AI平台，可以轻松地进行模型训练和部署。\n2. 提供了丰富的开发工具和库，包括Python、TensorFlow、PyTorch等热门AI框架。\n3. 支持多种硬件平台，包括CPU、GPU、TPU等。\n4. 支持多种操作系统，包括Windows、Linux、macOS等。\n5. 提供了一套完整的AI应用开发流程，包括数据预处理、模型训练、模型部署、模型优化等。\n\nModelBox的应用开发流程如下：\n\n1. 确定应用需求，包括输入数据、输出数据、模型类型等。\n2. 选择合适的开发工具和库，包括Python、TensorFlow、PyTorch等。\n3. 在ModelBox平台上构建AI应用，包括数据预处理、模型训练、模型部署等。\n4. 进行模型优化和调试，以提高模型性能和准确度。\n5. 将模型部署到目标硬件平台上，并进行测试和验证。\n6. 完成应用开发和部署，并进行维护和更新。\n\nModelBox可以帮助开发者快速构建、部署和管理AI应用，提高AI应用的开发效率和性能。', 'so

In [2]:
# 编写Gradio调用函数
import mdtex2html
from service.config import LangChainCFG
from service.configuration_chatglm import ChatGLMConfig


#将文本中的字符转为网页上可以支持的字符，避免被误认为是HTML标签
def parse_text(text):
    lines = text.split("\n")
    lines = [line for line in lines if line != ""]
    count = 0
    for i, line in enumerate(lines):
        if "```" in line:
            count += 1
            items = line.split('`')
            if count % 2 == 1:
                lines[i] = f'<pre><code class="language-{items[-1]}">'
            else:
                lines[i] = f'<br></code></pre>'
        else:
            if i > 0:
                if count % 2 == 1:
                    line = line.replace("`", "\`")
                    line = line.replace("<", "&lt;")
                    line = line.replace(">", "&gt;")
                    line = line.replace(" ", "&nbsp;")
                    line = line.replace("*", "&ast;")
                    line = line.replace("_", "&lowbar;")
                    line = line.replace("-", "&#45;")
                    line = line.replace(".", "&#46;")
                    line = line.replace("!", "&#33;")
                    line = line.replace("(", "&#40;")
                    line = line.replace(")", "&#41;")
                    line = line.replace("$", "&#36;")
                lines[i] = "<br>"+line
    text = "".join(lines)
    return text

#采用流聊天方式（stream_chat）调用模型，使得生成答案有逐字生成的效果
# def predict(input, chatbot, max_length, top_p, temperature, history, past_key_values):
#     chatbot.append((parse_text(input), parse_text(input)))
#     for response, history, past_key_values in model.stream_chat(tokenizer, input, history, past_key_values=past_key_values,
#                                                                 return_past_key_values=True,
#                                                                 max_length=max_length, top_p=top_p,
#                                                                 temperature=temperature):
#         chatbot[-1] = (parse_text(input), parse_text(response))

#         yield chatbot, history, past_key_values

# 在这里定义 application
config = LangChainCFG()
application = LangChainApplication(config)

def predict(input_text, chatbot, max_length, top_p, temperature, history, past_key_values):
    application.knowledge_service.init_knowledge_base()
    chatbot.append((parse_text(input_text), parse_text(input_text)))
    # chat_glm_model = ChatGLMForConditionalGeneration(config=ChatGLMConfig)
    
    response_dict = application.get_knowledeg_based_answer(parse_text(input_text), history_len=5, temperature=0.1, top_p=0.9, top_k=4, chat_history=history)
    if 'result' in response_dict and isinstance(response_dict['result'], str):
        result_text = response_dict['result']
        response = parse_text(result_text)
        chatbot[-1] = (parse_text(input_text), response)
        yield chatbot, history, past_key_values
    else:
        # Handle the case where 'result' key is missing or not a string
        # You can modify this part based on your requirements
        yield chatbot, history, past_key_values
    
    # for response, history, past_key_values in chat_glm_model.stream_chat(tokenizer, input, history, past_key_values=past_key_values,
    #                                                             return_past_key_values=True,
    #                                                             max_length=max_length, top_p=top_p,
    #                                                             temperature=temperature):
        
        

#去除输入框的内容
def reset_user_input():
    return gr.update(value='')

#清除状态
def reset_state():
    return [], [], None

Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.24s/it]
No sentence-transformers model found with name /home/ma-user/work/text2vec-large-chinese. Creating a new one with MEAN pooling.


In [3]:
#运行Gradio界面，运行成功后点击“Running on public URL”后的网页链接即可体验
import gradio as gr


with gr.Blocks() as demo:
    gr.HTML("""<h1 align="center">ChatGLM3-6B</h1>""")

    chatbot = gr.Chatbot()
    with gr.Row():
        with gr.Column(scale=4):
            with gr.Column(scale=12):
                user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=10,container=False)
            with gr.Column(min_width=32, scale=1):
                submitBtn = gr.Button("Submit", variant="primary")
        with gr.Column(scale=1):
            emptyBtn = gr.Button("Clear History")
            max_length = gr.Slider(0, 32768, value=8192, step=1.0, label="Maximum length", interactive=True)
            top_p = gr.Slider(0, 1, value=0.8, step=0.01, label="Top P", interactive=True)
            temperature = gr.Slider(0, 1, value=0.95, step=0.01, label="Temperature", interactive=True)

    history = gr.State([])
    past_key_values = gr.State(None)

    submitBtn.click(predict, [user_input, chatbot, max_length, top_p, temperature, history, past_key_values],
                    [chatbot, history, past_key_values], show_progress=True)
    submitBtn.click(reset_user_input, [], [user_input])

    emptyBtn.click(reset_state, outputs=[chatbot, history, past_key_values], show_progress=True)

demo.queue().launch(share=True, inbrowser=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a327a5f4636300ce08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)



#####init_knowledge_base#####

modelbox.jpg
modelbox.pdf
modelbox.txt
第一个应用.md

#####init_knowledge_base#####

modelbox.jpg
modelbox.pdf
modelbox.txt
第一个应用.md

#####init_knowledge_base#####

modelbox.jpg
modelbox.pdf
modelbox.txt
第一个应用.md
